# การติดตั้งและเริ่มใช้งาน Hyperledger Fabric เบื้องต้น

*** ตัวอย่างนี้เป็นการติดตั้งและทดลองรัน Hyperledger Fabric version อย่างง่าย ซึ่งไม่เหมาะแ่กการใช้งานใน production environment ***

## Requirements

1. ติดตั้งโดยเครื่องมือ สคริปต์ และตัวอย่างที่ทาง official Hyperledger มีให้
2. ติดตั้งบนเครื่อง Stand alone เพียงเครื่องเดียว
3. ติดตั้งโดยใช้ Docker เป็นหลัก
4. ใช้ตัวอย่าง chaincode example_02 ที่ official มีให้
5. ผู้ติดตั้งมีสิทธิ์รัน docker

## System requirements

1. OS: Ubuntu 18.04 LTS (MacOS/Windows ก็สามารถติดตั้งได้เช่นกัน แต่ต้องใช้ทักษะ config Docker เพิ่มเติม)
2. CPU: ยิ่งเยอะยิ่งดี เพราะ Blockchain มี service หลายตัวมาก รวมไปถึงใช้งาน database หลาย instance
3. RAM: อย่างน้อย 8 GB สำหรับการติดตั้งแบบเครื่องเดียว
4. Internet: ไม่เช่นนั้นจะไม่สามารถ run docker container ได้

## Step โดยรวมในตัวอย่างนี้ประกอบด้วย

0. Setup เครื่องสำหรับใช้งาน notebook นี้
1. Setup เครื่องสำหรับการติดตั้งและใช้งาน Hyperledger Fabric
2. Generate crypto materail ซึ่งเป็น artifact สำหรับการสร้าง blockchain network และ permissioned channel สำหรับการทำ transaction  ต่างๆ
3. deploy blockchain network และทดสอบด้วย end to end test ที่ตัวอย่างให้มา
4. terminate blockchain service 

สำหรับเครื่องที่ใช้ทดสอบตัวอย่างนี้มีสเปคดังนี้

In [1]:
lsb_release -a
lscpu | grep "Model name"
lsmem | grep "Total online memory"

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.3 LTS
Release:	18.04
Codename:	bionic
Model name:          Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz
Total online memory:       8G



# Step 0 Install & Setup environment สำหรับ notebook นี้

*** step นี้ให้ run แต่ละ command ใน terminal นะครับ ***

ติดตั้ง  [Bash kernel](https://github.com/Calysto/calysto_bash) สำหรับ Jupyter

```shell
pip install calysto_bash
```

แก้ให้ไม่ต้องใส่ password สำหรับคำสั่ง sudo ด้วยการ edit file /etc/sudoers

```shell
sudo visudo
```

ใส่

```text
username     ALL=(ALL) NOPASSWD:ALL
```

เข้าไปในบรรทัดสุดท้ายและ save file /etc/sudoers

เปิด notebook นี้ด้วยการรันคำสั่งใน terminal

```shell
sudo -E env "PATH=$PATH" jupyter notebook --allow-root
```

# Step 1 Setup เครื่องที่ใช้งาน

## ติดตั้ง docker engine และ docker-compose ต่างๆ

In [2]:
sudo apt-get install -y docker.io docker-compose




docker-compose is already the newest version (1.17.1-2).
docker.io is already the newest version (18.09.7-0ubuntu1~18.04.4).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.



## ตั้งค่าให้ user สามารถใช้ docker ได้โดยไม่ต้องใช้ `sudo` ([Docker post installation](https://docs.docker.com/install/linux/linux-postinstall/))


In [3]:
sudo groupadd docker
sudo usermod -aG docker $USER

groupadd: group 'docker' already exists



## ทดสอบการทำงานของ docker

In [4]:
docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/




# Step 2 Download Hyperledger Fabric binary files และ ตัวอย่าง

download file ที่จำเป็นในการใช้งาน โดยจะมีสิ่งที่ต้อง download 3 ประเภทหลัก คือ 

### 1. Hyperledger Fabric docker images
เป็น docker image ที่ pack core service ของ Hyperledger Fabric เอาไว้ ซึ่งสามารถ custom config/parameter ต่างๆได้
### 2. Hyperledger Fabric samples 
เป็นตัวอย่าง config และ chaincode (smart contract) ที่ทางทีมงาน Hyperledger เตรีมไว้ให้ทดลองเล่น และสามารถนำไปประยุกต์ใช้กับ app ของเราได้
### 3. Hyperledger Fabric platform-specific binaries
เป็น tool สำหรับการสร้าง crypto material และเรียกใช้งาน blockchain service ต่างๆ

คำสั่งที่ใช้ download ทุกอย่างแบบปกติ คือ

```shell
curl -sSL http://bit.ly/2ysbOFE | bash -s
```

step ข้างบนจะใช้เวลาค่อนข้างนาน เพราะต้อง download
1. Hyperledger Fabric docker images จำนวนมาก
2. Hyperledger Fabric samples หลายตัวอย่าง
3. Hyperledger Fabric platform-specific binaries ซึ่งใช้เวลา download ค่อนข้างนาน


เรามามารถ bypass ได้โดยการใส่ option

-d : bypass docker image download

-s : bypass fabric-samples repo clone

-b : bypass download of platform-specific binaries

ตัวอย่างการใช้งานเมื่อไม่ต้องการ download docker image และ samples

```shell
curl -sSL http://bit.ly/2ysbOFE | bash -s -- -d -s
```

ในตัวอย่างนี้จะทำการ download เฉพาะ binary file และตัวอย่างในเวอร์ชั่นเก่าเท่านั้น ซึ่ง docker image ที่จำเป็นจะถูก pull ในขณะ deploy service อยู่แล้ว

In [5]:
%%shell
## official scripts, however, i cannot download binaries with this script
# curl -sSL http://bit.ly/2ysbOFE | bash -s -- -d

# Changed script to my stable version
curl -sSL https://raw.githubusercontent.com/hyperledger/fabric/v1.4.3/scripts/bootstrap.sh | bash -s -- -d




Installing hyperledger/fabric-samples repo





===> Checking out v1.4.3 of hyperledger/fabric-samples


HEAD is now at f86ec95 [FAB-16390] Added filter for invalid transactions





Installing Hyperledger Fabric binaries





===> Downloading version 1.4.3 platform specific fabric binaries


===> Downloading:  https://nexus.hyperledger.org/content/repositories/releases/org/hyperledger/fabric/hyperledger-fabric/linux-amd64-1.4.3/hyperledger-fabric-linux-amd64-1.4.3.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current


                                 Dload  Upload   Total   Spent    Left  Speed




  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0

  0     0    0     0    0     0   


 44 6382k   44 2832k    0     0   136k      0  0:00:46  0:00:20  0:00:26  149k

 47 6382k   47 3000k    0     0   139k      0  0:00:45  0:00:21  0:00:24  151k

 50 6382k   50 3240k    0     0   142k      0  0:00:44  0:00:22  0:00:22  157k

 53 6382k   53 3416k    0     0   144k      0  0:00:44  0:00:23  0:00:21  162k

 55 6382k   55 3536k    0     0   143k      0  0:00:44  0:00:24  0:00:20  170k

 58 6382k   58 3720k    0     0   146k      0  0:00:43  0:00:25  0:00:18  187k

 61 6382k   61 3920k    0     0   146k      0  0:00:43  0:00:26  0:00:17  176k

 63 6382k   63 4064k    0     0   145k      0  0:00:43  0:00:27  0:00:16  157k

 66 6382k   66 4256k    0     0   149k      0  0:00:42  0:00:28  0:00:14  170k

 68 6382k   68 4392k    0     0   145k      0  0:00:43  0:00:30  0:00:13  154k

 71 6382k   71 4568k    0     0   147k      0  0:00:43  0:00:31  0:00:12  153k

 73 6382k   73 4680k    0     0   147k      0  0:00:43  0:00:31  0:00:12  150k

 75 6382k   75 4800k    0     0   146k 

[ข้อมูลการติดตั้งเพิ่มเติม](https://hyperledger-fabric.readthedocs.io/en/release-1.4/install.html)

Set system path เพื่อให้ระบบมองเห็น binary

In [6]:
export PATH=$PWD/bin:$PATH

# Step 3 Generate crypto material

change directory ไปที่ fabric-samples/first-network

In [7]:
cd fabric-samples/first-network

ลองดูว่าใน script first network สามารถทำอะไรได้บ้าง

In [8]:
bash byfn.sh

Usage: 
  byfn.sh <mode> [-c <channel name>] [-t <timeout>] [-d <delay>] [-f <docker-compose-file>] [-s <dbtype>] [-l <language>] [-o <consensus-type>] [-i <imagetag>] [-a] [-n] [-v]
    <mode> - one of 'up', 'down', 'restart', 'generate' or 'upgrade'
      - 'up' - bring up the network with docker-compose up
      - 'down' - clear the network with docker-compose down
      - 'restart' - restart the network
      - 'generate' - generate required certificates and genesis block
      - 'upgrade'  - upgrade the network from version 1.3.x to 1.4.0
    -c <channel name> - channel name to use (defaults to "mychannel")
    -t <timeout> - CLI timeout duration in seconds (defaults to 10)
    -d <delay> - delay duration in seconds (defaults to 3)
    -f <docker-compose-file> - specify which docker-compose file use (defaults to docker-compose-cli.yaml)
    -s <dbtype> - the database backend to use: goleveldb (default) or couchdb
    -l <language> - the chaincode language: golang (default) or node

Generate crypto material สำหรับ channel ชื่อ `getstartfabric`

In [9]:
# enter y to continue
echo "y" | 
bash byfn.sh generate -c getstartfabric

Generating certs and genesis block for channel 'getstartfabric' with CLI timeout of '10' seconds and CLI delay of '3' seconds
proceeding ...
/home/ratthapon/Blockchain/fabric-samples/first-network/../bin/cryptogen

##########################################################
##### Generate certificates using cryptogen tool #########
##########################################################
+ cryptogen generate --config=./crypto-config.yaml
org1.example.com
org2.example.com
+ res=0
+ set +x

Generate CCP files for Org1 and Org2
/home/ratthapon/Blockchain/fabric-samples/first-network/../bin/configtxgen
##########################################################
#########  Generating Orderer Genesis block ##############
##########################################################
CONSENSUS_TYPE=solo
+ '[' solo == solo ']'
+ configtxgen -profile TwoOrgsOrdererGenesis -channelID byfn-sys-channel -outputBlock ./channel-artifacts/genesis.block
2019-12-08 06:28:24.059 +07 [common.tools.configtxgen

ตรวจสอบไฟล์ที่ script gen มาให้

In [10]:
tree ./crypto-config

./crypto-config
├── ordererOrganizations
│   └── example.com
│       ├── ca
│       │   ├── ca.example.com-cert.pem
│       │   └── efd46609da859bba58893c07498c90dee40e709118c39d8ae2261a9de9381a4e_sk
│       ├── msp
│       │   ├── admincerts
│       │   ├── cacerts
│       │   │   └── ca.example.com-cert.pem
│       │   ├── config.yaml
│       │   └── tlscacerts
│       │       └── tlsca.example.com-cert.pem
│       ├── orderers
│       │   ├── orderer2.example.com
│       │   │   ├── msp
│       │   │   │   ├── admincerts
│       │   │   │   ├── cacerts
│       │   │   │   │   └── ca.example.com-cert.pem
│       │   │   │   ├── config.yaml
│       │   │   │   ├── keystore
│       │   │   │   │   └── 4eed5a7a6a431d2bb13ca5a19cf382ae4f15f85689d15a680ad002ba976dae09_sk
│       │   │   │   ├── signcerts
│       │   │   │   │   └── orderer2.example.com-cert.pem
│       │   │   │   └── tlscacerts
│       │   │   │       └── tlsca.example.com-cert.pem
│       │   │   └── tls
│       │   │  

In [11]:
tree ./channel-artifacts

./channel-artifacts
├── channel.tx
├── genesis.block
├── Org1MSPanchors.tx
└── Org2MSPanchors.tx

0 directories, 4 files



# Step 3 Deploy Hyperledger Fabric Network

clean docker volume & image ก่อนทำการ deploy service

In [12]:
docker kill $(docker ps -q -f name=example.com) $(docker ps -q -f name=cli)
docker system prune -f
docker volume prune -f

"docker kill" requires at least 1 argument.
See 'docker kill --help'.

Usage:  docker kill [OPTIONS] CONTAINER [CONTAINER...]

Kill one or more running containers
Deleted Containers:
a4e787a6315d05190b5e1668686d6115ef97a7de04fed759a22dbf90ee0853f4
52b78c6a180933224ce8784f8e00641482856f54ffc3d65f1d9a453ecbefcaa2

Total reclaimed space: 0B
Total reclaimed space: 0B



In [13]:
echo "y" | 
bash ./byfn.sh up -c getstartfabric

Starting for channel 'getstartfabric' with CLI timeout of '10' seconds and CLI delay of '3' seconds
proceeding ...
LOCAL_VERSION=1.4.3
DOCKER_IMAGE_VERSION=1.4.4
=================== WARNING ===================
  Local fabric binaries and docker images are  
  out of  sync. This may cause problems.       
Creating network "net_byfn" with the default driver
Creating volume "net_peer0.org2.example.com" with default driver
Creating volume "net_peer1.org2.example.com" with default driver
Creating volume "net_peer1.org1.example.com" with default driver
Creating volume "net_peer0.org1.example.com" with default driver
Creating volume "net_orderer.example.com" with default driver





Creating peer0.org2.example.com
Creating peer0.org1.example.com
Creating peer1.org1.example.com
Creating peer1.org2.example.com
Creating orderer.example.com

Creating cli
CONTAINER ID        IMAGE                               COMMAND             CREATED                  STATUS                  PORTS              

ตรวจสอบการ deploy ว่าสำเร็จหรือไม่จาก log ที่ออกมา

ตรวจสอบ service ที่ถูก deploy ว่ามีอะไรบ้าง

In [14]:
docker ps

CONTAINER ID        IMAGE                                                                                                  COMMAND                  CREATED             STATUS                  PORTS                      NAMES
fbf88c3eb635        dev-peer1.org2.example.com-mycc-1.0-26c2ef32838554aac4f7ad6f100aca865e87959c9a126e86d764c8d01f8346ab   "chaincode -peer.add…"   1 second ago        Up Less than a second                              dev-peer1.org2.example.com-mycc-1.0
29a73802c5a4        dev-peer0.org1.example.com-mycc-1.0-384f11f484b9302df90b453200cfb25174305fce8f53f4e94d45ee3b6cab0ce9   "chaincode -peer.add…"   5 seconds ago       Up 4 seconds                                       dev-peer0.org1.example.com-mycc-1.0
3043228091dd        dev-peer0.org2.example.com-mycc-1.0-15b571b3ce849066b7ec74497da3b27e54e0df1345daff3951b94245ce09c42b   "chaincode -peer.add…"   8 seconds ago       Up 7 seconds                                       dev-peer0.org2.example.com-mycc-1.0
ad2c153f1e

# Step 4 Terminate Hyperledger Fabric Network

ทำการ shutdown และ clean docker container ต่างๆด้วย

*** ระวัง คำสั่งนี้จะทำการ kill docker container ทั้งหมดในเครื่อง ****

In [15]:
echo "y" |
bash ./byfn.sh down -c getstartfabric

Stopping for channel 'getstartfabric' with CLI timeout of '10' seconds and CLI delay of '3' seconds
proceeding ...












Removing network net_byfn
Removing volume net_peer0.org3.example.com
Removing volume net_peer1.org3.example.com
Removing volume net_orderer2.example.com
Removing volume net_orderer.example.com
Removing volume net_peer0.org2.example.com
Removing volume net_peer0.org1.example.com
Removing volume net_peer1.org1.example.com
Removing volume net_peer1.org2.example.com
Removing volume net_orderer5.example.com
Removing volume net_orderer4.example.com
Removing volume net_orderer3.example.com
fbf88c3eb635
29a73802c5a4
3043228091dd
Untagged: dev-peer1.org2.example.com-mycc-1.0-26c2ef32838554aac4f7ad6f100aca865e87959c9a126e86d764c8d01f8346ab:latest
Deleted: sha256:71a46eb7b2062f663d835dba23b60e9c51b86933040dc9c06cb464e4e512da2c
Deleted: sha256:60f8796515107af6a31d75774f847ea819ea8f2946fbfa5a5066d472b3382514
Deleted: sha256:7ada5ddb7c1d18111ff31ded6e4722b8ae8b5381e96e78d76e